In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
sys.path.append('/content/drive/My Drive/Colab Notebooks/federated_learning')

!pip install shap==0.40.0

# Experimental Setup

In [ ]:
from federated_learning.utils import SHAPUtil, experiment_util, Visualizer
from federated_learning import TestClientPlane, ClientPlane, Configuration, ObserverConfiguration
from federated_learning.server import Server
from datetime import datetime
from federated_learning.nets import FMNISTCNN
from federated_learning.dataset import FMNISTDataset
import os
config = Configuration()
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = FMNISTDataset
config.MODELNAME = config.FMNIST_NAME
config.NETWORK = FMNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.datasetObserverConfiguration = "MNIST"
neutral_label = 2

In [ ]:
# Google Colab Settigns
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"

In [ ]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
visualizer = Visualizer(shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)

In [ ]:
import numpy as np
import copy
import torch
import os
for j in range(3):
    for i in range(100):
        experiment_util.set_rounds(client_plane, server, i+1)
        experiment_util.run_round(client_plane, server, i+1)
    file = "/content/drive/My Drive/Colab Notebooks/temp/models/ex8/{}_FMNIST_round_{}_.model".format(j, i+1)
    if not os.path.exists(os.path.dirname(file)):
            os.makedirs(os.path.dirname(file))
    torch.save(server.net.state_dict(), file)

In [ ]:
def cos_similarity_values(s_client, s_server):
    from scipy import spatial
    import numpy as np
    cos_similarity = [[] for i in range(10)]
    differences_sum = [[] for i in range(10)]
    shap_subtract = np.subtract(s_client, s_server)
    for row_idx, row in enumerate(s_client):
        for img_idx, image in enumerate(row):
                cos_similarity[row_idx].append(spatial.distance.cosine(image.flatten(),s_server[row_idx][img_idx].flatten()))
                differences_sum[row_idx].append(np.sum(shap_subtract[row_idx][img_idx].flatten()))
    return np.sum(cos_similarity), np.sum(np.abs(differences_sum)), np.max(cos_similarity), np.abs(np.array(differences_sum).flatten()[np.argmax(np.abs(np.array(differences_sum).flatten()))])
    


# Test Client

In [ ]:
config.NUMBER_OF_CLIENTS = 40
config.FROM_LABEL = 0
config.TO_LABEL = 1
test_client_plane = TestClientPlane(config, observer_config, data, shap_util)
experiment_util.update_configs(test_client_plane, server, config, observer_config)

In [ ]:
import torch
for j in [100]:
    data = config.DATASET(config)
    test_client_plane = TestClientPlane(config, observer_config, data, shap_util)
    model_file = file = "/content/drive/My Drive/Colab Notebooks/temp/models/ex8/1_FMNIST_round_{}_.model".format(j)
    server.net = FMNISTCNN()
    server.net.load_state_dict(torch.load(model_file))

    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    server_shap = server.get_shap_values()

    config.POISONED_CLIENTS = 20
    experiment_util.update_configs(test_client_plane, server, config, observer_config)
    print(len(test_client_plane.clients[0].train_dataloader.dataset.dataset.targets[test_client_plane.clients[0].train_dataloader.dataset.dataset.targets == 5]))

    test_client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(test_client_plane, config, 100)
    poisoned_clients = experiment_util.select_poisoned(test_client_plane, 100)
    clean_dis = []
    poisoned_dis = []
    clean_diff = []
    poisoned_diff = []
    clean_max_diff = []
    poisoned_max_diff = []
    clean_max_dis = []
    poisoned_max_dis = []
    print("Clean")
    print(len(test_client_plane.clients[0].train_dataloader.dataset.dataset.targets[test_client_plane.clients[0].train_dataloader.dataset.dataset.targets == 5]))
    for idx, i in enumerate(clean_clients[:20]):
        test_client_plane.update_clients(server.get_nn_parameters())
        test_client_plane.clients[i].train(j+1)
        clean_client_shap = test_client_plane.clients[i].get_shap_values()
        distance, diff, dis_max, diff_max = cos_similarity_values(clean_client_shap, server_shap)
        clean_dis.append(distance)
        clean_diff.append(diff)
        clean_max_dis.append(dis_max)
        clean_max_diff.append(diff_max)
        if (idx+1)%25 == 0:
            print(clean_dis[idx-25:idx])
    
    clean = {
        "dis" : clean_dis, 
        "diff" : clean_diff,
        "max_dis" : clean_max_dis,
        "max_diff" : clean_max_diff
    }

    print("Poisoned")
    server.net =  FMNISTCNN()
    server.net.load_state_dict(torch.load(model_file))
    for idx, i in enumerate(poisoned_clients[:20]):
        test_client_plane.update_clients(server.get_nn_parameters())    
        test_client_plane.clients[i].train(j+1)
        poisoned_client_shap = test_client_plane.clients[i].get_shap_values()
        distance, diff, dis_max, diff_max = cos_similarity_values(poisoned_client_shap, server_shap)
        poisoned_dis.append(distance)
        poisoned_diff.append(diff)
        poisoned_max_dis.append(dis_max)
        poisoned_max_diff.append(diff_max)
        if (idx+1)%25 == 0:
            print(poisoned_dis[idx-25:idx])
    print(len(test_client_plane.clients[poisoned_clients[0]].train_dataloader.dataset.dataset.targets[test_client_plane.clients[poisoned_clients[0]].train_dataloader.dataset.dataset.targets == 5]))
    test_client_plane.reset_default_client_nets()
    test_client_plane.reset_poisoning_attack()
    
    poisoned = {
        "dis" : poisoned_dis, 
        "diff" : poisoned_diff,
        "max_dis" : poisoned_max_dis,
        "max_diff" : poisoned_max_diff
    }

In [ ]:
import pprint
pp = pprint.PrettyPrinter(width=6000, compact=True)
pp.pprint(clean)

In [ ]:
import pprint
pp = pprint.PrettyPrinter(width=6000, compact=True)
pp.pprint(poisoned)

# Client

In [ ]:
config.NUMBER_OF_CLIENTS = 200
config.FROM_LABEL = 7
config.TO_LABEL = 4
client_plane = ClientPlane(config, observer_config, data, shap_util)
experiment_util.update_configs(client_plane, server, config, observer_config)

In [ ]:
import torch
config.FROM_LABEL = 7
config.TO_LABEL = 4
for j in [100]:
    data = config.DATASET(config)
    client_plane = ClientPlane(config, observer_config, data, shap_util)
    model_file = file = "./temp/models/ex8/2_MNIST_round_{}_.model".format(j)
    server.net =  MNISTCNN()
    server.net.load_state_dict(torch.load(model_file))

    server.test()
    recall, precision, accuracy = server.analize_test()
    print("Original", recall, precision, accuracy)
    server_shap = server.get_shap_values()

    config.POISONED_CLIENTS = 100
    experiment_util.update_configs(client_plane, server, config, observer_config)
    print(len(client_plane.clients[0].train_dataloader.dataset.dataset.targets[client_plane.clients[0].train_dataloader.dataset.dataset.targets == 5]))

    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, 100)
    poisoned_clients = experiment_util.select_poisoned(client_plane, 100)
    clean_dis = []
    poisoned_dis = []
    clean_diff = []
    poisoned_diff = []
    clean_max_diff = []
    poisoned_max_diff = []
    clean_max_dis = []
    poisoned_max_dis = []
    print("Clean")
    print(len(client_plane.clients[0].train_dataloader.dataset.dataset.targets[client_plane.clients[0].train_dataloader.dataset.dataset.targets == 5]))
    for idx, i in enumerate(clean_clients[:20]):
        client_plane.update_clients(server.get_nn_parameters())
        client_plane.clients[i].train(j+1)
        clean_client_shap = client_plane.clients[i].get_shap_values()
        distance, diff, dis_max, diff_max = cos_similarity_values(clean_client_shap, server_shap)
        clean_dis.append(distance)
        clean_diff.append(diff)
        clean_max_dis.append(dis_max)
        clean_max_diff.append(diff_max)
        if (idx+1)%25 == 0:
            print(clean_dis[idx-25:idx])
    
    clean = {
        "dis" : clean_dis, 
        "diff" : clean_diff,
        "max_dis" : clean_max_dis,
        "max_diff" : clean_max_diff
    }

    print("Poisoned")
    server.net =  MNISTCNN()
    server.net.load_state_dict(torch.load(model_file))
    for idx, i in enumerate(poisoned_clients[:20]):
        client_plane.update_clients(server.get_nn_parameters())    
        client_plane.clients[i].train(j+1)
        poisoned_client_shap = client_plane.clients[i].get_shap_values()
        distance, diff, dis_max, diff_max = cos_similarity_values(poisoned_client_shap, server_shap)
        poisoned_dis.append(distance)
        poisoned_diff.append(diff)
        poisoned_max_dis.append(dis_max)
        poisoned_max_diff.append(diff_max)
        if (idx+1)%25 == 0:
            print(poisoned_dis[idx-25:idx])
    print(len(client_plane.clients[poisoned_clients[0]].train_dataloader.dataset.dataset.targets[client_plane.clients[poisoned_clients[0]].train_dataloader.dataset.dataset.targets == 5]))
    client_plane.reset_default_client_nets()
    client_plane.reset_poisoning_attack()
    
    poisoned = {
        "dis" : poisoned_dis, 
        "diff" : poisoned_diff,
        "max_dis" : poisoned_max_dis,
        "max_diff" : poisoned_max_diff
    }

In [ ]:
import pprint
pp = pprint.PrettyPrinter(width=6000, compact=True)
pp.pprint(clean)

In [ ]:
import pprint
pp = pprint.PrettyPrinter(width=6000, compact=True)
pp.pprint(poisoned)

# Results for Testing 
20 clean 20 poisoned

In [ ]:
# 40 clients with dataset of size 250
clean = [
    #0: 8->5
    {'diff': [2.527527323204996, 2.1214496958106834, 1.875232576781153, 2.201806024548462, 1.9607416181106507, 1.8916114025722357, 1.7568974215301896, 1.201721460222492, 1.6643706394639826, 1.3382407830582492, 1.919471344366982, 1.472800261780769, 1.8526794671425306, 1.8904876163301776, 1.5305153071327693, 1.885291460750688, 3.532104676864502, 1.7308657206420164, 1.505059249208026, 2.2120554843863864], 'dis': [13.624510311893221, 17.806227738122548, 11.54317147508228, 17.28607316092144, 18.8753210705613, 13.44664759739775, 8.510489877865659, 7.789161578029692, 11.741070544749812, 9.57700825670158, 14.533838598074196, 10.442905327933353, 14.743529905167467, 12.997588466860448, 8.363599098876119, 10.519592473425012, 23.151445889691672, 13.733714634113486, 12.03723316701147, 17.920163161303506], 'max_diff': [0.28706374464302575, 0.2297951694702931, 0.2569825607920371, 0.2572014208340456, 0.18425588789135983, 0.2477462047331791, 0.18203682768617213, 0.15950651545550665, 0.32915983890915324, 0.12253617692163754, 0.19804235301570206, 0.185558849197064, 0.3339367493992149, 0.14739756559639838, 0.2476057216851335, 0.18267799284539255, 0.3722398166719876, 0.17539241646467207, 0.11480440585066243, 0.31277188248468324], 'max_dis': [0.4055812517840576, 0.615376920600569, 0.35162867734566805, 0.608112225515191, 0.5605168791315667, 0.31589320137260524, 0.25616181099032465, 0.2755852350184277, 0.34755049192881693, 0.22197749312411097, 0.4212742500501112, 0.38096208523509556, 0.5140105423629662, 0.4603161014587902, 0.19547712626264468, 0.32781313069371165, 0.6949831846296034, 0.43435744468289295, 0.5496392119290159, 0.3935743643392219]},
    #0: 7->3
    {'diff': [1.642031581326696, 1.743018058760786, 1.936567904279339, 1.5551069145574996, 1.8468902573725388, 2.917925038630668, 1.4946105954242226, 1.837489266879035, 2.1123225359729583, 1.2926249829324719, 1.5687449837780099, 1.837214554250617, 1.8311512689551936, 1.601520188368946, 2.6263547382961328, 1.0203183071487778, 1.8963454669816662, 1.5011004188976926, 1.4403517450779961, 3.942496320880773], 'dis': [19.340732044130306, 10.14811877467391, 12.70619001113882, 9.087726282188344, 11.523806504545629, 22.045961760152107, 12.487803346546112, 11.332821843617806, 13.604610523257668, 9.394272299462195, 11.225253685859762, 11.815107350362432, 12.76856167231432, 13.731745595999408, 17.90656608304638, 6.4830848119257, 11.453579351517538, 15.53178934536953, 10.074378400509595, 24.275725397023624], 'max_diff': [0.1410826163878216, 0.2967350486768785, 0.25230616985671794, 0.14875746740468632, 0.2123230205600768, 0.38607111874585665, 0.1707907741826551, 0.21109715122105488, 0.2316903293527064, 0.20437787700973548, 0.2775584304698768, 0.13870900408509496, 0.19640375831077805, 0.142641480537814, 0.2841901011272666, 0.0797308313578794, 0.17387805640383652, 0.20818188471954713, 0.24312361690444462, 0.32096607147827383], 'max_dis': [0.46466521734003774, 0.3242086450788624, 0.49968014403132854, 0.25340906553583753, 0.28510485980296185, 0.5169410175792114, 0.48279316156474805, 0.3052250542499778, 0.39459510472899306, 0.1585399804263149, 0.45957969943073984, 0.3557020739883191, 0.31851602808680335, 0.3156802071950531, 0.4241959269105381, 0.10947695877985353, 0.28970379247641764, 0.5293261434774392, 0.22237612242693905, 0.6972387603227255]},
    #1: 1->2
    {'diff': [1.943829202488801, 1.3314509614048875, 2.2615599843256895, 2.2570779605893394, 1.6008585427090474, 2.0154949841258873, 1.6737279357172068, 2.281181291572858, 1.6448912099068689, 1.4624621503471562, 2.2026429304788477, 1.5211200770319158, 2.1530602194308672, 1.6733637903244287, 1.6652177528542365, 2.8209232548104315, 1.9917234624126896, 1.8717947696037052, 1.795715642489354, 2.291344921515497], 'dis': [12.30249379962572, 11.315581591078306, 13.561355455867107, 17.91634337247759, 13.006086219555476, 17.317734778904, 9.807377301302001, 13.92643961720287, 11.411426700089915, 9.651059070645823, 15.623075417836148, 9.911030498515863, 14.209803882921745, 10.810984298838942, 11.763047232457252, 20.74892411796387, 17.972291527362287, 11.15633911601878, 15.902965719200935, 18.309551112392334], 'max_diff': [0.2945748487726325, 0.09950986050174127, 0.22410595492220242, 0.2860857298664916, 0.1698988291415935, 0.19461326112465027, 0.22357813094939605, 0.4064024660215184, 0.1577843351127699, 0.14354507583737863, 0.3021147308792944, 0.1535180963527507, 0.27003431241117365, 0.1501962723085235, 0.28450062716584545, 0.2678440178258219, 0.23082007233653812, 0.19762628210692768, 0.1639323082497255, 0.339116566348886], 'max_dis': [0.3265837637860449, 0.24346927647926508, 0.34652854760510754, 0.5827020444409876, 0.43595152992583985, 0.4337149508696566, 0.22538532607131134, 0.6721045534285992, 0.21915892910564594, 0.3176207041934914, 0.4345110642772082, 0.2786557403840042, 0.3360829992706108, 0.3124357879910664, 0.27091245418521626, 0.5320419803013536, 0.38650605738637456, 0.24814632994277785, 0.3760032033158728, 0.5673575292599378]},
    #1: 6->4
    {'diff': [2.044927181337801, 1.9564569654895743, 1.6196569070757663, 1.78303240043266, 1.1306204932109847, 1.6046491982920794, 1.5284754912181233, 2.489089541501608, 1.545010007515449, 1.3703604634552726, 1.2943434571174997, 2.3551066176738757, 2.0116931626378776, 1.4955025814856362, 1.3272270750753443, 1.4499545290865645, 2.0531197395975083, 1.5089597162829993, 1.448306407099249, 1.255180168335589], 'dis': [21.591894543800912, 16.857237642820802, 14.548576563887364, 13.043946302242185, 10.075215419066039, 16.938516422661525, 14.745361654610136, 19.167836452588254, 13.686457888356598, 12.667897102967213, 16.507206440641074, 22.614131776939402, 13.841574978825811, 18.247949751369433, 16.99497107100515, 14.931863957846419, 18.01044820792071, 10.894110593505623, 10.501211531079399, 11.272236651468578], 'max_diff': [0.20422196069356136, 0.3347465239102845, 0.1813465689104916, 0.2799113970323477, 0.09900656249478601, 0.14092646986150054, 0.18653182239185284, 0.4108304149697588, 0.14554643704008532, 0.15572450205054134, 0.07655406935663578, 0.40709428488653254, 0.3308275705483519, 0.10957242106647591, 0.19511477344664918, 0.20452211728474903, 0.21554192642356007, 0.15884446961758147, 0.18420784854018013, 0.16427169702087951], 'max_dis': [0.6051399082646317, 0.4012695710166345, 0.3927065004379757, 0.31447259137969674, 0.3417758888716953, 0.5047653550855704, 0.4802362200796316, 0.3769719150433065, 0.32182818033481597, 0.31297434229989707, 0.42983632469717137, 0.6461305630044228, 0.26258802274167525, 0.6216758158780139, 0.5539610462559486, 0.36294799226138985, 0.4668807407419947, 0.2577167366939631, 0.2521341947706073, 0.31060031715366887]},
    #2: 0->1
    {'diff': [1.5983280050105526, 2.606717268622316, 1.217665824316533, 1.517867647439979, 1.4642255003069884, 1.041036970504609, 1.6199416555674386, 1.421214532470297, 1.6375749674044136, 1.694721200655547, 1.8009959960842423, 1.1826170792341908, 2.3734976375539043, 1.9797612284399344, 1.9357311729470759, 1.9072239976523992, 1.3949532423099944, 1.412743468456853, 1.521563259539142, 1.485953966528455], 'dis': [14.083483052934447, 20.26793719222015, 8.711658197321515, 11.922032874704811, 14.332959031311995, 10.70208663793118, 17.923383845617614, 14.136029792333753, 11.719767867923691, 11.571605757665633, 19.115817883245896, 12.393227502105962, 19.613745990666075, 17.527357288043543, 14.113520340890647, 19.30267164813045, 14.242163967811825, 9.195216899268067, 14.735945186793208, 12.577505068692098], 'max_diff': [0.19391900164543152, 0.45061005220045613, 0.16382589984119147, 0.1746242042088202, 0.21044744820293326, 0.13355357772493295, 0.17299232446553736, 0.13955961935704023, 0.2575727745490899, 0.41486652199649776, 0.15130607962758802, 0.16382280648142, 0.4300891251508293, 0.23128911412813696, 0.2917042819468687, 0.26239182124799065, 0.1678225342081463, 0.15835660186775247, 0.16825822444878646, 0.18235307447944127], 'max_dis': [0.495504945313226, 0.472073592924967, 0.24987942282742437, 0.26933944981414537, 0.42405305562805606, 0.2591626234420582, 0.5249150152482756, 0.6737637439649361, 0.4086424357430273, 0.2700495639281334, 0.6283688550027249, 0.4050370404204504, 0.496671405131641, 0.6949389710366274, 0.30102357756903353, 0.6088737037746739, 0.28196446928258334, 0.17908622752482883, 0.48578369154266343, 0.3615289756262231]},  
]

In [ ]:
poisoned = [
    #0: 8->5
    {'diff': [2.110356208280102, 2.3598775569881187, 2.1714350908728464, 2.344293799447971, 2.392749544731219, 2.7180052082279227, 2.814945924050667, 2.4273295046567203, 2.5142249993448846, 2.02554054283536, 2.4602717231258726, 2.6021343551899467, 3.0860334182067213, 2.635838749584821, 2.30967192589047, 3.780229053493713, 2.845259806196272, 2.7909479922761453, 2.4106836667852916, 2.464815857239843], 'dis': [26.954282823637286, 25.70867901118856, 21.294702388390043, 25.12372263634412, 23.18850596302251, 25.34312572966773, 28.582500040235068, 23.29631528735492, 24.818577128489586, 20.276081641614777, 26.298536086374153, 26.641424620966557, 30.708042228299398, 25.017708061439208, 22.764939014705718, 28.915895318715346, 23.870062729647096, 23.821602406465704, 24.90010074308783, 24.2819914763234], 'max_diff': [0.22861708977007392, 0.2896344398151691, 0.24372963927289248, 0.3311864207594917, 0.36697016680001027, 0.27987567531051805, 0.32464588338599953, 0.21500776805755795, 0.2611963542273892, 0.3217138793107688, 0.2315646426874405, 0.22627968787713273, 0.34690759220633205, 0.25668416144347517, 0.3255089929393957, 0.33793449695994826, 0.3490231197920539, 0.22545190963762085, 0.21150531574741496, 0.22304488634458844], 'max_dis': [1.2152481919955496, 0.9448154913322876, 0.7904666464986416, 1.104087402268558, 1.171423454955948, 0.9322516576852667, 1.2230794222339174, 0.7941423985469716, 0.8817018316753145, 0.9237292636093545, 1.0075035440317655, 0.9259279975082503, 1.23014893251873, 0.9014032576611963, 0.8982588135794454, 1.1327001953011, 0.7551667439572447, 0.9146498660820779, 0.963568461174067, 0.9496890342281382]},
    #0: 7->3
    {'diff': [2.781343311259533, 2.4783432243772365, 2.2143465248794247, 3.5192339084149165, 2.282768495547682, 3.0077563376568097, 4.099196577200708, 4.343738953782717, 2.1852118976476618, 2.859943631977858, 2.8142961768884005, 2.6364472599746254, 4.842880769269579, 3.283859564458727, 3.4333931528612354, 3.079073375653572, 3.6627841466732622, 3.4911245422355615, 3.04363428790295, 3.1710579450162153], 'dis': [25.423242010730863, 22.447760783782687, 26.300857008068895, 28.696028939680623, 26.91962011320447, 32.06197141302034, 32.655378375890706, 30.236441479624695, 24.878709669875153, 24.819389559322126, 27.361970461413808, 24.447249931719778, 41.102101946162534, 29.68325597272674, 27.7766999446346, 27.21837816828641, 30.344404376155904, 29.212728299529562, 30.594011577856733, 29.724276706220873], 'max_diff': [0.28020181446257103, 0.2813679248101615, 0.15153894226209497, 0.26035913447187164, 0.17822590501582813, 0.3472346261341225, 0.3851258049974071, 0.37779885461767027, 0.17928108312159974, 0.3057484700133478, 0.21664159537000627, 0.22439442616639357, 0.38657653895674504, 0.32458438867184103, 0.33509215384578234, 0.22072562378422678, 0.2623404235511795, 0.2753991916241053, 0.2884816871203154, 0.3468507596472957], 'max_dis': [0.8686713788002443, 0.8542186284239137, 0.9092995935908561, 0.8337160606345392, 0.8593434910925193, 0.9194659561525089, 0.9080943103909198, 0.878905586216627, 0.8429566297629906, 0.8747668101526409, 0.8947816874704868, 0.9866907673904532, 0.9853019968584943, 0.7691216445146902, 0.9025444626515323, 0.9187774788171715, 0.9583660633419573, 0.807708213126787, 0.8613913431334803, 0.8781497044536812]},
    #1: 1->2
    {'diff': [3.0460885240814135, 2.975943268308939, 4.0602052023054656, 3.46311148868188, 4.4393047590991825, 3.7519031319834495, 2.811992831519249, 2.9870360043141093, 3.853888986308186, 3.371694782554065, 3.047390294437893, 3.02437832081248, 3.1795571664813194, 3.587464970781886, 3.237772109861051, 3.6137523724723497, 2.7782174887135334, 3.172989131525224, 3.212325294836806, 3.617333078624872], 'dis': [24.62985738479408, 27.330599659259796, 32.46629566348377, 26.483391294009273, 33.51051662891458, 29.496873804224172, 26.094023613230153, 26.70860955727008, 32.87214464437973, 27.376382345285744, 22.68792334591387, 27.860546247632637, 24.61404196151494, 26.432749541088494, 27.444157575694398, 27.129871827152442, 30.76173886045651, 25.738378196887478, 26.419121052073535, 27.723430597442032], 'max_diff': [0.2657621698895607, 0.2789658470977939, 0.42371073855312513, 0.2563366495284791, 0.3505271478523766, 0.3766698637365193, 0.3356004330790119, 0.43017792681478095, 0.3180582785498154, 0.3406102958726532, 0.2797930604242609, 0.22912494904923175, 0.2793007847123368, 0.27828067276286683, 0.30611122407205904, 0.3560018500225559, 0.28445198339977323, 0.2381527199969618, 0.30986373208855916, 0.39612137143707393], 'max_dis': [0.7651668750534125, 0.8115570882755381, 0.7823133607409611, 0.7805773032595531, 0.7687692204686468, 0.7775992978507508, 0.7145724182079054, 0.7592677148266763, 0.8748573929850468, 0.810981826906689, 0.7413323132682367, 0.7598041435021691, 0.7778172790198153, 0.7547901511756813, 0.7605703292416438, 0.7742052968802813, 0.7752315633712789, 0.7692198937579017, 0.7826807799613585, 0.7848081452047408]},
    #1: 6->4
    {'diff': [2.5262721715198175, 2.9499799994669282, 2.432195147672416, 2.404895982844909, 2.3350670183674183, 2.5492736115528754, 2.26935837831397, 4.03361027895716, 2.228213474669216, 2.667404524697403, 2.3286622377458857, 3.0294618910797855, 2.498032421261872, 2.181633039429519, 2.1373209257304158, 2.945119715147092, 2.4338082982518214, 2.8519987917716065, 2.2074037518124143, 3.196109032834241], 'dis': [20.969661878713186, 23.624570235083446, 21.368272372280035, 23.35339431913948, 21.608979933122352, 18.182535298137275, 25.12379605342835, 36.01228181110485, 23.247699591600753, 28.46464355741929, 20.02790896253129, 23.84974386436552, 20.79446417568417, 22.289293384168626, 21.947601182408057, 29.98505106861463, 26.598881809951887, 28.83146891696296, 20.346085060241823, 24.39743903922228], 'max_diff': [0.2942847325501763, 0.32349677525081777, 0.4116144421078931, 0.3044219270387418, 0.23918417888065324, 0.3151200713627249, 0.2522085676353738, 0.29210714891757106, 0.24166753533246155, 0.2984004884041882, 0.3075312769601828, 0.37464745003529254, 0.29608528473701146, 0.23692823321424616, 0.2613781918449183, 0.4130559765068509, 0.28024555514536154, 0.34025218975374977, 0.28823595180733985, 0.4218970061673142], 'max_dis': [0.5466400608102309, 0.5196663919644232, 0.5671593624833401, 0.8166039535114055, 0.5830197104876456, 0.5240035497817046, 0.7640922497625117, 0.9180879703044393, 0.6873076912312642, 0.5885141937491576, 0.6766029935743333, 0.7093385666915404, 0.6929191196728116, 0.7646081924660035, 0.6472159685617886, 0.89459907736683, 0.8301267847635765, 0.794759662601272, 0.6001682831284695, 0.6442427013425726]},
    #2: 0->1
    {'diff': [3.0634139650850587, 2.7732153971458775, 4.2265490947406485, 2.698146238106938, 2.726014884262179, 3.0557758508284323, 2.8552955150950616, 3.194731073712286, 3.2497385486569836, 2.6819724754062317, 3.5318203683460965, 2.7687698394949556, 2.55193941089206, 2.646218912458159, 2.4711571915811232, 2.4652180937616235, 2.404041343565897, 3.0273220648249843, 2.2424216885796024, 3.5409629940183143], 'dis': [32.683558332232714, 35.57297494020974, 35.23647372750045, 24.255328424298916, 27.592591084854874, 23.051253501973644, 27.812723409311722, 33.13765097835202, 32.48156022051113, 31.75903849039718, 33.61616040246486, 26.669027469962653, 23.503970270501327, 27.959458731421442, 29.373946250542296, 28.173134049530926, 27.493966843223795, 31.00854234070804, 27.450487119129317, 34.52421007414121], 'max_diff': [0.37205660863272483, 0.24525630133668241, 0.5088666711020782, 0.24291891876117133, 0.25201929254233146, 0.41453637854577075, 0.22818655180049546, 0.3830644453495835, 0.3270753478820616, 0.24766508451243852, 0.25957032203839514, 0.28438347443096923, 0.28030795235944006, 0.25028680043343915, 0.24692526910553703, 0.3513080945952398, 0.2506833173110984, 0.3787245152544054, 0.24875924156423546, 0.3261032373942494], 'max_dis': [0.8884419440570511, 1.0008472131293484, 0.9222374766929788, 0.8247886601567044, 0.8990700602874003, 0.6837549687958809, 0.7473061333360735, 0.9020499138600699, 0.9574468288209322, 0.8173382442188053, 0.8808803097248097, 0.9071566499433565, 0.7929911405658605, 1.0240204311899685, 0.7800594338763794, 0.8380355079660239, 0.8317839341672001, 0.9346410492291394, 0.8823153730543941, 0.8084095338128809]},

]

In [ ]:
clean_test = []
clean_test_label = []
for idx, data in enumerate(clean):
    for i, d in enumerate(data['diff']):
        clean_test.append([data['diff'][i], data['dis'][i], data['max_diff'][i], data['max_dis'][i]])
        clean_test_label.append(0)

In [ ]:
poisoned_test = []
poisoned_test_label = []
for idx, data in enumerate(poisoned):
    for i, d in enumerate(data['diff']):
        poisoned_test.append([data['diff'][i], data['dis'][i], data['max_diff'][i], data['max_dis'][i]])
        poisoned_test_label.append(1)

In [ ]:
X_test = clean_test + poisoned_test

In [ ]:
y_test = clean_test_label + poisoned_test_label
print(y_test[:5])

In [ ]:
from sklearn.utils import shuffle
X_test, y_test = shuffle(X_test,y_test, random_state=0)
print(y_test[:5])

## Train Preperations

id: 0 -> 1,3
id: 1 -Y 5,4
id: 2 -Y 8,9

In [ ]:
clean = [
    {
        'diff': [0.8962569583697961, 1.4530295128344415, 1.4212203955352143, 1.7019163920565195, 1.862944493521555, 1.401084345688276, 1.2712609809674138, 2.0965895998886706, 1.8308236421266597, 1.8245732441111244, 1.8337222885971052, 2.178871509841429, 1.4647270494975901, 2.814849894596636, 2.2823008204331643, 1.6401242793461708, 0.772876363910081, 1.3242825210995506, 1.858862092410938, 1.8675694866228227, 2.0491559156548718, 1.8127167737710608, 1.2039529690771804, 1.7966534075304788, 1.267558346815904, 1.9382423515641705, 1.77412609220545, 2.3488481924202405, 2.0897521285371425, 2.2060258623311326, 1.435234062196882, 1.792667435156968, 2.5928438611268847, 1.2034716972566921, 1.7501934102193037, 1.760340877408659, 1.8736662557120944, 1.453405494683689, 1.5365037723385206, 1.6860487056563627, 1.998897067519476, 1.1068156055178742, 1.4221080736377598, 1.8139034556600158, 1.6974026528976653, 1.4340933661688962, 1.6535269804558654, 1.2713221510024477, 2.306038266011362, 1.6977204412081077, 1.2303289204515928, 1.4780976464155273, 1.9413483630666737, 1.1245931402291656, 1.6965356298096814, 1.150516623218146, 2.5568614769226983, 1.2664393075016842, 1.9751210671631743, 2.1828929577290896, 2.1026654687638984, 1.3677431987664306, 1.7032867749349392, 1.2226785169791639, 1.6778423644293559, 2.10624187359996, 1.516957194845621, 2.201371099819018, 2.7797515023108885, 1.250748768986073, 1.8310652263995864, 2.268143638642915, 1.892448541770848, 2.081776559719005, 1.526013697759048, 2.790088919068865, 2.1705960864275475, 1.2966379401444053, 2.3573409088478092, 1.5778747207091108, 1.2649814839707927, 1.719269040422733, 1.6084895538626398, 1.734016057430272, 1.9957612759080448, 1.6293511813405417, 2.1915180986541447, 2.3722347480040242, 1.3838188942050165, 2.9900199572661843, 1.8280032615664803, 1.524656764760207, 1.902256063383986, 0.8751697538188282, 1.5811963412243417, 1.958240142840685, 1.6319472028479018, 1.305091181302385, 1.7252242215110503, 1.0851338199816354],
        'dis': [9.99598678772894, 10.518830386814669, 11.668979900144462, 10.72861410971739, 11.09394270649212, 14.322875726350592, 9.271901885106315, 13.43674712628381, 12.681371262203774, 11.826370750851536, 16.826949918229477, 14.841030879243396, 12.602313967769753, 17.599969248049714, 11.995959709130755, 12.734888711771367, 8.70728510727852, 14.0324992139522, 13.891523992574706, 15.325041210194877, 15.9346775115954, 15.950372122444175, 9.043477595679857, 11.044428730769589, 12.999438991730738, 13.128966478616928, 12.935598804380936, 18.012597515482188, 10.849999302419992, 15.91614722654045, 8.79619171307941, 13.07863933376833, 20.82094986004721, 10.718447932383057, 10.16845583283848, 10.529454156285473, 11.4704077337462, 8.24957408975533, 10.960852729731444, 12.29675765784889, 14.576605583658171, 9.372142825882678, 13.323543430845096, 14.786507958941135, 15.966571251751539, 9.4544057240735, 11.553159759810113, 9.234664640091033, 16.229537445251246, 10.606551278992452, 9.548959280135566, 8.896405967666048, 12.51066189821927, 6.5679207283497405, 15.30835227825933, 9.859027993517193, 13.022387627085514, 10.082271868127522, 15.981778135051332, 14.709403058571176, 14.497275128268098, 12.770819718249843, 18.12982873888616, 7.65076679256433, 14.527369530727556, 16.15172422692629, 12.052659511286564, 14.078898696399797, 15.052545066368364, 9.734992229397424, 12.905241090899391, 12.081409054062666, 10.859200667555482, 13.419033676548462, 15.156205176095854, 16.70565252495428, 15.710404650077843, 13.721162138309815, 12.270296025681285, 9.880817722592296, 10.050477761862075, 13.798223028532783, 8.793733977024875, 13.01505471816258, 18.17103454089263, 10.18473663979901, 11.92794814835138, 15.083636401154509, 9.998951072964813, 21.943370881236902, 12.572862249949127, 10.95768538172218, 13.039682273181226, 6.263158763404303, 10.269386353007599, 12.231130087895897, 11.359050820126962, 10.452366563010377, 9.451458233378634, 8.335563072665025],
        'max_diff': [0.0813449288074315, 0.15127775689124923, 0.13475386657141286, 0.24898422799651865, 0.1891083594942491, 0.16901317170376984, 0.16101904063272876, 0.3423597668097237, 0.30649763010251574, 0.2648846103660353, 0.14625057275319198, 0.2772543173563329, 0.13058781532729669, 0.27507728089911865, 0.31245215972483376, 0.15439750932145868, 0.09515148747171365, 0.1630717111645943, 0.19886205616731611, 0.31366530140213555, 0.1477129027698204, 0.22152088378673818, 0.08304311987001878, 0.3024030866365326, 0.08781741292555714, 0.22660211642039796, 0.12229202870580302, 0.1606214019458266, 0.346132604644398, 0.21363298236791894, 0.16980276031162056, 0.19157483496400773, 0.23122194564499776, 0.19062638401120358, 0.2577247391214281, 0.26756076156166664, 0.3112447233379727, 0.17613038183343432, 0.10888523934407979, 0.11896468247281378, 0.2730456380122206, 0.08894529208357149, 0.1460079058471866, 0.2057158414970237, 0.27216564711231683, 0.20532233131286493, 0.12626183176515898, 0.11389396768649473, 0.32223308070072854, 0.295310064346528, 0.11102985672927601, 0.14531950799760318, 0.290850356983948, 0.13716456907962993, 0.15388940840546095, 0.11986348807891112, 0.27279742366899473, 0.09670853548341707, 0.22179402975986662, 0.32663688917278844, 0.26012201489565245, 0.14422227500659687, 0.14129594001876264, 0.11380113318885421, 0.19342183273349578, 0.3086210477879277, 0.1301088086833525, 0.23543924626762636, 0.29799585327920064, 0.13917475486914208, 0.20552686863507796, 0.17300144821807661, 0.1808188197125773, 0.27224773494962706, 0.13216007980876965, 0.3227802236130284, 0.2127556146249061, 0.13764390308853613, 0.2953816137707037, 0.16796672399389356, 0.09166776433836943, 0.15902859987802143, 0.18520096957397358, 0.2795384433236404, 0.2723937174340327, 0.28419604681135, 0.327329711070405, 0.26385057089207653, 0.1270001417061456, 0.3511118041256598, 0.12098757783521885, 0.09935487924093156, 0.22049218431773931, 0.12072610356912161, 0.2357261997374014, 0.15122815234434062, 0.1384293554420133, 0.1127296251197355, 0.28188117634789656, 0.1305772708330366],
        'max_dis': [0.19555272323289596, 0.3509416707755235, 0.3088987872967206, 0.2989676789740874, 0.31258870720859855, 0.2892519378514893, 0.20702680549135166, 0.385640243279993, 0.31913454801536145, 0.4449586705987131, 0.49207777629253735, 0.3121956719820038, 0.31680125890788213, 0.4658046949623146, 0.26111279905303986, 0.3521509626689484, 0.22209720222708795, 0.4042438312864268, 0.4941329898217969, 0.5343920373440667, 0.5172727974276408, 0.3587800474068512, 0.23707310076657362, 0.3729430762467988, 0.3972041987343593, 0.3603374246066079, 0.5252206979476646, 0.7565104798582337, 0.2760266502249641, 0.38785349511747, 0.16361415638223875, 0.3477803085825877, 0.591098999197966, 0.21407535510203823, 0.3098036792585047, 0.4860382915493118, 0.34852852440608273, 0.19479572755588892, 0.3783588725564305, 0.27855680320794685, 0.3861139246809957, 0.3275039501657643, 0.2581959910188937, 0.375092543082006, 0.6448022639703275, 0.24419977306423535, 0.2132498078602817, 0.2097885336992633, 0.5652473606600881, 0.36107961587895265, 0.2857404377956405, 0.19771336938810746, 0.3519303642532786, 0.1701042921131497, 0.5162231238906744, 0.24974149306169613, 0.3703567621792544, 0.21670078011285698, 0.48919978453114277, 0.6365292469895265, 0.4024110587345133, 0.3591277175234736, 0.5092150902656096, 0.30704171014288795, 0.327138884206967, 0.6216793885350973, 0.3662156565465332, 0.31628720602193894, 0.4035101720228256, 0.2163140168420371, 0.29244867206936387, 0.3782397643737647, 0.28144303211130517, 0.5599975080348516, 0.45833091791720704, 0.39871278759388573, 0.3665765599966271, 0.4007951935564654, 0.3940150716710663, 0.20371331393571945, 0.26870050415082003, 0.5296615466280002, 0.1554414082131601, 0.34460800465033825, 0.5992836287336885, 0.26304394532467235, 0.36135329707436337, 0.3768648291851765, 0.2404663844106193, 0.6091737061679108, 0.27574591769368084, 0.23401748894211816, 0.35789786625647824, 0.11831405056808508, 0.2541793378535633, 0.3028724937542957, 0.21567378153641714, 0.24311170679530103, 0.23569395585729325, 0.23534968589697303]
    },
    {
        'diff': [2.184212435809473, 2.69867229252141, 1.6490115499572746, 1.1864327249045075, 2.4603908712050373, 1.7235230974512963, 1.036687037552976, 3.0550538155500395, 1.7430777679618858, 2.3055889795666946, 1.6658322350326504, 1.940883129800859, 1.434532231625068, 0.9399518700137464, 1.7949566055460033, 2.123406821254371, 2.0009358220896196, 1.105828937239722, 2.0203831831050754, 1.0447781339229263, 2.25047227436405, 1.4561292129694958, 1.9540483523781047, 1.4883825035982245, 2.5822220119174095, 1.8843911335680528, 2.4839026940353466, 2.384820460899178, 1.7445243121983498, 2.568028979904171, 2.301925814141312, 2.244852292609254, 2.753684238604733, 1.7185868776522057, 1.6797416151650124, 2.146465785999218, 1.331179022228193, 1.3581076584349734, 1.1380053637118026, 1.9255932287146589, 2.0007922181711377, 2.6509525197422144, 1.9539156085013956, 2.021723797609784, 1.5585289940435316, 0.9336194949595886, 2.8962443856432474, 2.4342128020418956, 1.5470417028573935, 1.799131042460968, 1.9275525108509945, 1.52934010761637, 1.7168571120479201, 1.431752969501363, 1.4728214755359137, 1.0654500369118032, 1.7788081025914981, 1.8712430915958782, 1.2441550921669702, 1.1639252417022157, 1.5674972891369479, 1.2648878270470603, 2.101969765019755, 1.2835041739532855, 1.3571758075513791, 1.1141325775891309, 1.6115985028130912, 2.2246647393716583, 2.063497297742936, 1.8185691522934966, 1.9318563201852625, 2.591717497783646, 2.42539265909111, 2.02156858438624, 1.9988821482733343, 1.838101817997954, 1.746496376034931, 2.1275911108585297, 1.6317988979346558, 1.9253988969284352, 1.3230422189383626, 2.0843111645237578, 1.154468279454499, 3.643435079658146, 1.8618545348131732, 1.1039962724236658, 1.345476366160323, 1.4442713610998001, 2.7617039854593823, 1.6961009673025707, 1.9072337996733537, 1.4300500136758276, 1.026134248826722, 2.2846825908381545, 2.0874544739271506, 2.1444986580770875, 1.6643972461185088, 1.8983623244522563, 2.4085297683152898, 1.3005975054342598],
        'dis': [16.516172588033857, 20.635462323466438, 12.88401944581362, 8.68108587330977, 13.456799180763216, 12.50254656628498, 8.235285140149896, 22.514608996104858, 11.018339718309834, 13.850615963614706, 10.729173094657144, 14.819661072394897, 12.24018547051392, 8.697005379201375, 9.999430767244286, 11.865170621639804, 17.041871825049533, 8.259737924414937, 15.735119078005606, 8.13375598043855, 16.823997513941972, 11.61117016398574, 11.011836940742471, 11.18702117161435, 15.500773183255312, 11.568779337761015, 17.365773436381403, 14.403161146075995, 15.147522987982574, 15.730641405714971, 16.69696559472621, 15.540448538253973, 17.868188539680823, 15.555955293944674, 11.153245228744423, 12.34109741522675, 10.052647532906605, 11.336956300813345, 7.6436815713324595, 12.76683755967523, 12.310663163444133, 18.752198987199776, 12.384048588674384, 13.525905246743887, 8.373634077451747, 9.901466816518884, 21.733563280988086, 14.766340091229777, 13.666293057392739, 16.878097174640406, 13.122060147654578, 12.631072780530188, 12.41485425962711, 8.269532421756212, 10.901371358557533, 7.595956165365248, 10.16681761122225, 14.19471395881408, 9.829647791052167, 9.663417681304166, 12.8980652957201, 14.037537027114642, 14.647384996097331, 8.834054961244634, 12.119550278261146, 8.015417966341326, 13.03713354547426, 16.98492515883917, 12.73413746152786, 14.428447081246404, 15.006890866902127, 16.37800427584101, 12.193251511973356, 12.894848564982638, 12.39994022838858, 15.083013504561322, 12.168306310976893, 18.401754983762444, 12.91468922942338, 14.891683697709714, 7.557205449561701, 11.89358310871825, 13.75087577870688, 25.227499209485426, 15.417042977773146, 10.763027520990404, 14.03506509354851, 13.585913824675426, 15.788727170705751, 13.710671689361556, 13.96742373155532, 10.075136040672305, 8.656757937009466, 20.918199386631755, 15.073016865441957, 13.227492181298686, 11.403873878606264, 14.03934236919423, 16.097988536298608, 8.874674299578128],
        'max_diff': [0.274576411401922, 0.36513826561438023, 0.33766639042093965, 0.17851957350046077, 0.21268095352991845, 0.13693001878779498, 0.11882903678535628, 0.3450225335223286, 0.1993284679737234, 0.24760723999222156, 0.17101836578602003, 0.1816487885228355, 0.09712542646502564, 0.08215306386708743, 0.3245425504512416, 0.3062238751591808, 0.1324430431446908, 0.07242446377677103, 0.17609633382000434, 0.09108418763729614, 0.3113409414344118, 0.14457291949230577, 0.16422550558471616, 0.23474175918505757, 0.2955753946422828, 0.23244092407396844, 0.2531897920978202, 0.32170334530201594, 0.28699188379024143, 0.2610397643506088, 0.29859208902935386, 0.4040799861718236, 0.37533567866053175, 0.10275936639548133, 0.19743621666699349, 0.29244656380096656, 0.13499151237237483, 0.13729885012760712, 0.14317236543721612, 0.2294226642057028, 0.1934920820589019, 0.306174712916206, 0.36635828837451534, 0.27014131336017877, 0.2107193383596826, 0.09427666568379867, 0.3440271768906169, 0.27869704301413356, 0.16590644816871847, 0.19329726679099124, 0.14193653368870285, 0.2577948496680724, 0.29745164633018106, 0.1281896722129927, 0.14565447028579948, 0.10242378778735528, 0.231763379598954, 0.3865354276450659, 0.12114248341863776, 0.13309519609085385, 0.11634701569905559, 0.1088141113409824, 0.27831772970276814, 0.18896653800679886, 0.20664669402738234, 0.20247483515709308, 0.22383670784697873, 0.1649501236357509, 0.22764950042189946, 0.3136256248463331, 0.14971661160857686, 0.33530074977235813, 0.3561189425417015, 0.22540231743899464, 0.23075259414006155, 0.31668331234444036, 0.1467044821086274, 0.22459513232588868, 0.12922799342368307, 0.2894161757586209, 0.1529407640897541, 0.21983834012047493, 0.07142771999216091, 0.35164373372187707, 0.13310639226782595, 0.08053325012329426, 0.10538602546699671, 0.12304111277296592, 0.2520509759655485, 0.15192382383054975, 0.18912558512786837, 0.10243485396644147, 0.09130450427574954, 0.20281411257152726, 0.21611986171215758, 0.2665929504647404, 0.12433112191298079, 0.29949716187878606, 0.23631870163082647, 0.22884737489764895],
        'max_dis': [0.4644889750539315, 0.6440303276672544, 0.34760956453363523, 0.24967270163278743, 0.3402641252090277, 0.23687976458272564, 0.24454547784683955, 0.6327063293462538, 0.38529209450774693, 0.31525896196207437, 0.24158666825930109, 0.5026982595503378, 0.40888506216300147, 0.1827171123614202, 0.22914059612017534, 0.38691376482109485, 0.3846044641992553, 0.17674415083749628, 0.4658988705705851, 0.1772930213529743, 0.5625760460473743, 0.25568851464804654, 0.27124425468545643, 0.22561794441679717, 0.28211415250309224, 0.30260925887835355, 0.5675073214370124, 0.3091968275539363, 0.3844857352595228, 0.3895895403912015, 0.5124546897139202, 0.37432931631657307, 0.45406335346457516, 0.6162585311437885, 0.283912172421819, 0.5160111460689327, 0.250254717004767, 0.3308546742466286, 0.20109308164841533, 0.2844106345781692, 0.44284663811152036, 0.5312873729822507, 0.3081895607053572, 0.3856699043974702, 0.18218497116566978, 0.24913331437274533, 0.6308202087689072, 0.6143760569262806, 0.4939961162035348, 0.6449597082772411, 0.4474596471867739, 0.37581692055924376, 0.5161658190836509, 0.23859829428246138, 0.3292816021543189, 0.27728292798035137, 0.23055214111889621, 0.4430984209117673, 0.33049925143075143, 0.21659310752470062, 0.3448126232822676, 0.3880331322865209, 0.6276945265458785, 0.2570623636616015, 0.2772634455532229, 0.20154828319133133, 0.30242869820860474, 0.6737248632490853, 0.46236182206311105, 0.39425061080563883, 0.37102255854454036, 0.3371887423190435, 0.35081675112582855, 0.28631278519371983, 0.4559481329459658, 0.345382022438826, 0.41866956143160095, 0.5906316009931395, 0.31939156840109706, 0.47240597331055034, 0.22641241757735842, 0.2378996133532688, 0.32861589869027763, 0.6823063847816879, 0.34125358541388084, 0.35639964062493545, 0.47628181086799315, 0.29727901385441013, 0.4538738538931285, 0.4276398733298019, 0.31697901540363216, 0.19621168692321012, 0.18891638086862794, 0.7175244884588465, 0.344925030230126, 0.4497737415103086, 0.29154944334516075, 0.4608588275912151, 0.4847179069001797, 0.2520141656806525]
    },
    {
        'diff': [1.8386950074593902, 1.6634315324992712, 1.7488162317754616, 2.1142141621030572, 2.2953470037436867, 1.554837594918989, 1.8925777341770662, 2.298578243657974, 1.630968166281211, 2.0231063986479922, 1.22933839663702, 2.6202497450762463, 2.004410906557121, 1.4429814370112142, 2.1481418413336457, 1.282967014271461, 1.4001429159598537, 1.886615436862944, 2.052534866956433, 2.3663806712166426, 0.9561461650295242, 1.9464050654491476, 1.9368922964866442, 1.6865057572122892, 1.2184466278025652, 1.5502113171057084, 1.2983852986846998, 1.725461581765086, 1.8587090603744005, 0.81889806439934, 1.7796916652814654, 2.3506146347113015, 2.3370235552128067, 2.0580356290266146, 2.019553371163496, 2.216606642162654, 2.23622887041271, 2.5366258318309094, 1.5107873105315721, 1.527121692155176, 1.1387424064117886, 2.0364743125496427, 1.1303759552418848, 1.778786293078344, 1.608275641032482, 2.2488070894891705, 1.8608442583440499, 1.3191727006148668, 1.2019391266704045, 2.385865901283838, 1.5965891880179262, 2.0464910095644058, 1.555273084461261, 1.5479257207304578, 1.3916285512606956, 1.644050139265124, 1.8621921727991864, 1.7766941687455768, 1.4728936459464845, 2.596042369911367, 1.1017936832221689, 1.3279372113898187, 1.998518908924, 1.5551058673081122, 2.2606263690889357, 1.7333245935302626, 1.2546864127570672, 2.5307279276066694, 2.2505484065909185, 2.202113300663852, 1.4968057011264069, 1.9517063939510453, 2.3322564122206546, 1.2643639360300094, 1.4703179638805433, 2.38414098302821, 2.699609317477572, 2.659496974571114, 0.8301182754483474, 1.6874184510529615, 1.9228633439410865, 1.7682307758377644, 1.3454053397968597, 1.3290714871175575, 1.8991405322658748, 1.3537703422510001, 1.3724369327106392, 1.2677793654226797, 1.9651791056342303, 1.8556718167544175, 2.605073323374646, 2.2997498000566403, 1.588602296762975, 2.302640987588432, 1.921415121486236, 2.557058798458388, 1.6011455175211917, 1.8645963707982818, 1.917576333819007, 2.3242858338741463],
        'dis': [12.753760726363181, 11.909328911621714, 12.647940557438597, 11.589115660923925, 15.82507108537933, 14.247273055697358, 12.821642190755, 17.586286308578504, 12.311885034712835, 12.70011052841767, 13.751287004574126, 13.572115761417741, 14.394513493698948, 9.85886799558657, 14.64262289469788, 12.172241426431045, 9.130419186934123, 14.815513474893008, 15.697913980572046, 18.73696559608703, 8.076063938201735, 16.69204695638927, 16.128235158601928, 11.507876278820492, 9.720042344837571, 11.538250934151106, 12.562173375206633, 10.548665856449258, 16.385408192884622, 9.812669231133837, 9.187757022514736, 18.69546988893741, 13.073412497877959, 15.008006103602112, 13.192176424800477, 19.025498668426334, 14.259107675202282, 20.63423886687827, 12.227423898611631, 8.821595442722273, 6.1070982148962525, 16.27762610855034, 9.806883241179133, 9.27055257178257, 13.481742307777843, 11.332103598572976, 10.917224930827624, 10.556088640264878, 9.348766536346353, 19.7983990090634, 14.255291379248359, 16.479813872910622, 12.383009688949953, 13.179835223605501, 9.865724064685066, 15.304421790760223, 10.305033284340453, 16.20702404863152, 11.791876880899762, 17.753536009484243, 9.427753211370979, 10.276278766159711, 15.899201831021614, 9.697312895962146, 19.598731485228218, 13.389337692241735, 9.492585067099988, 12.675009617136766, 14.44816489580994, 18.73760289152858, 12.254279316941775, 11.269730412198998, 13.707716641046721, 10.201896749954651, 13.420178170772571, 17.605843097503232, 17.398526282314098, 19.576742914587516, 8.726731653514962, 13.648445685434437, 16.2126602843441, 11.165991276135069, 9.665683188454208, 9.111249187055527, 14.344743818817356, 8.934122493063406, 8.18202786181925, 9.76089269991162, 9.960737393503267, 17.293736142815316, 15.475567778109236, 13.01054352752245, 12.933901621500365, 17.035863841291423, 13.68666366520209, 15.841371433712162, 11.276980526892398, 10.79917765842498, 12.162667939475076, 13.391094277353814],
        'max_diff': [0.23605650327532413, 0.18799124873091155, 0.16048691008315324, 0.2668155247636097, 0.2901236626562832, 0.175126558850617, 0.29844490467384777, 0.2057916724243486, 0.2373774231578878, 0.38761999057261853, 0.1033731141870815, 0.2763187359770667, 0.2636314471016543, 0.19979068434213243, 0.28190884326178534, 0.09246936517315185, 0.2232132616554361, 0.21069833189670106, 0.14031451257595862, 0.38535781299909067, 0.07657168018028382, 0.1848450775460836, 0.2740050486848311, 0.1564266771617513, 0.18425478014780317, 0.15830963596669667, 0.103255794376936, 0.1484568621519864, 0.1289105119193863, 0.10177480946570949, 0.15489992195448732, 0.16512457124018987, 0.3183237003266889, 0.20250171863485025, 0.3360540078898069, 0.20756263303577427, 0.2785474885642, 0.3194448363672884, 0.1823969333596311, 0.18853645349080317, 0.16581527196528612, 0.34074063614087535, 0.10622180389420699, 0.28520480608384624, 0.16465442188188018, 0.28888551699000464, 0.3135216016888078, 0.1505587057149096, 0.09421239667342718, 0.20936426016093546, 0.13092293087710005, 0.19267127848643772, 0.14441466522266944, 0.11502962454360544, 0.1478013797128236, 0.1376300127421306, 0.2008784635684412, 0.2933025289275307, 0.33366864497108173, 0.3871405345779513, 0.08637783941059862, 0.11493239746034334, 0.2968923659040793, 0.19538836243489044, 0.15392017571719208, 0.13247657387764633, 0.17697130753097035, 0.2869680719728631, 0.32909468317288937, 0.4120460619401265, 0.1330589389627126, 0.35667070214849694, 0.1832279275513624, 0.17317369231966478, 0.13178161163062718, 0.33473077211018953, 0.29142638746839555, 0.2806530182901401, 0.06145625464652249, 0.11838942135804043, 0.21536367873992635, 0.15994961903797034, 0.13295745563681294, 0.3082098180303152, 0.2142058741916698, 0.13544241670530655, 0.21120644211209916, 0.12787210609998745, 0.2742889570955698, 0.17753830644317414, 0.398448725533437, 0.40068961218193255, 0.2915010624783403, 0.29504948637036676, 0.2489056179146356, 0.29141937821535, 0.29330457984417, 0.23352715267470536, 0.19716003380006697, 0.21317258498901825],
        'max_dis': [0.5517996284956027, 0.346139999220763, 0.3578150248746027, 0.27758261067230316, 0.5950660862682897, 0.32849121893457434, 0.27834482256464765, 0.52217465415256, 0.3327107940737092, 0.4786452092536787, 0.4093820099684603, 0.3420958740519012, 0.3994797013618693, 0.23687372401221685, 0.425708064149527, 0.5497605564022754, 0.22029221675624522, 0.6047583424213605, 0.4065601259741487, 0.4383591959365125, 0.17650469262417223, 0.5149313867338609, 0.5527798908477184, 0.4648554941081704, 0.3385559870074475, 0.34595377963585183, 0.3247641737062824, 0.2856765654615838, 0.502562780810752, 0.20352857224076282, 0.18530713980634073, 0.6032536768323563, 0.3129240628501636, 0.4571137161624811, 0.4184689576167705, 0.5675545686675423, 0.3182111119915525, 0.6736220125645362, 0.3642959890355695, 0.2659877964821121, 0.13885747575596508, 0.5492107285007483, 0.23660323217042467, 0.2384142559917185, 0.30485940008542045, 0.288534926895891, 0.34627382602373813, 0.3443817785029899, 0.2976637854732055, 0.5918503305344893, 0.3422461645175445, 0.6161596687779463, 0.36701930781863756, 0.3977867277262649, 0.21977580774152805, 0.3901991848002898, 0.34903787257299757, 0.42456978744548757, 0.32408352041129673, 0.4787477164154299, 0.19070381951291915, 0.2260727782320502, 0.3069892491212468, 0.24497823558355558, 0.5244045247876462, 0.3168586357631473, 0.23825241890268545, 0.29143152284773466, 0.35704579580247364, 0.6146390306398442, 0.35564198698898664, 0.3034908766479677, 0.2871672454012051, 0.25646556198601633, 0.41369968699209514, 0.5170832430673751, 0.4200170551759068, 0.5610200419481539, 0.2795749950088249, 0.45718074181234925, 0.500209061903224, 0.3837948288079184, 0.24761545350894387, 0.2202603903682343, 0.40032317826814867, 0.3117144979946316, 0.1951924548908487, 0.360616553178587, 0.3653815519795339, 0.4944185603015283, 0.5744678922274701, 0.28477482775098284, 0.4596859882415354, 0.5059499221162089, 0.4340304239660676, 0.5143361319229665, 0.5269970756153279, 0.28380533464959123, 0.4146936723920661, 0.42870051555661526]
    },
]

In [ ]:
poisoned = [
    {
        'diff': [2.770378395329413, 2.0472128629400625, 2.5417623773534577, 2.634090644608809, 2.200435971418212, 3.0356335656608904, 1.9717888007703577, 2.3239919930746646, 2.226589463230373, 2.025615094440048, 2.8136040372482762, 2.3070462779283716, 1.8586679584015633, 2.1433013327995947, 2.778342028999953, 2.929286231170592, 2.338332794304163, 1.5175179666226666, 2.31509671214793, 1.7875860520254903, 2.6238799761502465, 2.229581899724873, 1.9730926076371424, 2.087290471342739, 1.7870038922814242, 1.780497044560808, 3.875350980726257, 1.9991682706882195, 2.7411737292198284, 2.071999804343752, 2.729636529650318, 2.670738576620696, 2.5392631983697598, 2.582867447515515, 1.8232728551956932, 2.7402418817155776, 2.580182905564119, 2.3224428116656224, 2.3670741038517438, 1.6596577810676674, 2.27247651065736, 2.478719566934106, 2.61676877851979, 2.156636051731575, 2.675711155148446, 2.120816927677293, 2.6334865886536276, 1.9204249415026065, 2.3336350730329003, 2.567201572114732, 1.9684725674898307, 3.47874195443459, 1.501253403455134, 2.3033735935358637, 1.6742251617822677, 1.8919382266583282, 1.6200656163530898, 2.147491152218051, 2.7860854253576224, 2.2123534398483526, 2.138048904635387, 1.6476528002021587, 1.7056580296757466, 2.9870452924179993, 2.19338748505495, 2.521123473389226, 2.669620956445949, 2.495255863753281, 2.390066729944834, 2.4548694132038906, 2.1518935756039754, 2.185314107003298, 1.9072568828998318, 3.752567539365934, 2.9546199212012767, 3.14851946340563, 2.957006936913836, 2.330728127812441, 2.8142932508924567, 1.9912237566561655, 2.19860271384116, 2.3217428390117996, 2.3272784933376895, 1.7881265090457683, 3.219281730892598, 2.6835749438985377, 2.0162068402341857, 2.974920682523244, 2.6162945185900286, 2.7112404378259307, 2.151821647312027, 3.1862134726773323, 2.8847877193088407, 2.482518931527469, 1.8846497241716125, 3.1511257969244655, 2.193760391089419, 2.324121896509199, 2.628036659428468, 2.239017927176569],
        'dis': [22.730410153492066, 28.16327704085318, 22.004395766922716, 21.868001692827548, 22.08876108533474, 22.7023648836059, 21.784820656495146, 22.3187020758246, 24.939240784498573, 21.87166345713098, 26.37060548577278, 18.86884845627312, 21.70442836411548, 20.575726228962296, 21.195428237924833, 27.246714283532164, 21.644953949942696, 18.026456119656242, 22.514835822614597, 18.561283635041047, 21.6279898536007, 23.074792202510707, 19.352802796574142, 18.783533605411847, 19.763423049945864, 19.7242377204433, 32.80283765242489, 27.309467939085437, 21.923458738212865, 26.45299546536427, 23.383944226285525, 23.799784404085194, 23.42823939362428, 24.907659418676456, 19.009954042026855, 24.74324072020374, 28.225872657162135, 19.806999796638042, 25.433079570416233, 18.55236593816474, 24.02016468368518, 20.692415928891247, 22.227900889719596, 22.33773703828694, 22.699548094481937, 24.35877086390477, 24.985788315760196, 19.87226432002107, 23.73364703942309, 29.11516993860885, 18.413199306004604, 29.08685800822088, 16.913864053276235, 21.512609116768488, 18.0242221380292, 17.722906888090762, 17.455360822475264, 20.290422542546796, 26.327275533529402, 20.92537058612355, 24.89336497068282, 18.703975102915386, 18.5360935403581, 21.459235211535237, 21.934007278690885, 23.07710097834514, 23.424563647798337, 20.879229683961324, 24.93624784871698, 21.663130612699142, 19.19270860932501, 20.23709368425466, 20.399539353590338, 27.416756788363813, 32.172966155113215, 27.085960827798548, 25.16827525177784, 30.230265209933325, 26.144762403905574, 21.89514399070828, 22.116249835479703, 21.891442246264447, 18.95916249311499, 18.247001344450084, 29.65700271328241, 23.491619651439585, 19.5614459235849, 25.148492296441624, 21.297976508646563, 26.574099561645795, 21.749222658315837, 26.35274059941388, 24.69117279383065, 19.894189875341336, 20.310678951772953, 28.154623973921264, 21.995726139740793, 21.25222538871312, 23.747503990157572, 24.656076745736144],
        'max_diff': [0.3918091736232334, 0.2248251957533511, 0.2758092804189289, 0.29999547638186275, 0.202181747878724, 0.29088417803881317, 0.21092879056396008, 0.2762431723887726, 0.29190340974728446, 0.21802579701037383, 0.3143020116050625, 0.2645272373268218, 0.22723552480227105, 0.2073289173043944, 0.27335587757527313, 0.28110985876672867, 0.2952235519060109, 0.18305372196367786, 0.16466176929184306, 0.19103251933912496, 0.2998601510177381, 0.1882688635675857, 0.2491771469809414, 0.2130955018290901, 0.1856049974665465, 0.1572445467125243, 0.30226298573700383, 0.15721126591240897, 0.2893330108801625, 0.24304682175575776, 0.23910514437716202, 0.24468416778897217, 0.2294243731714669, 0.2842285027720612, 0.18968239336286918, 0.2921408401498047, 0.20908185589519057, 0.2248513111426893, 0.34455967275643706, 0.2108869446336908, 0.23860926694053952, 0.2516422916274686, 0.3497720929257184, 0.2554169382612616, 0.20813044996336627, 0.1639865256178596, 0.2628948800736203, 0.22435138756714124, 0.20107965175517906, 0.23464742897884694, 0.2462911477625791, 0.33868783308168393, 0.18091392529846057, 0.24726454083869243, 0.21217700977601095, 0.2396512869179943, 0.22298488619534448, 0.21642177977313515, 0.22914108657854793, 0.20734381701389945, 0.2071879412408968, 0.20947686048157244, 0.234710293540364, 0.2907247644481661, 0.30405341564067245, 0.26324527059666547, 0.315929403227976, 0.30531648393808436, 0.1479735915537268, 0.2643039560757967, 0.20164485390291081, 0.1498753661118628, 0.21550079305771486, 0.3255756354370516, 0.20396529582541353, 0.2819526033829671, 0.27692167196638073, 0.200182190422865, 0.16833863249713044, 0.2438209798269202, 0.21938489393984573, 0.22454640554617455, 0.24188127134853166, 0.15919717148101148, 0.2611054324225829, 0.2935719178576335, 0.263566962298439, 0.2690048522906905, 0.27178367212324217, 0.2581876517092432, 0.23681436899155917, 0.29236350570997405, 0.25449795283273313, 0.2975155409113204, 0.22123963119928303, 0.23359009750172777, 0.26462485767107147, 0.21133251959101665, 0.31374636781386217, 0.12600278971706586],
        'max_dis': [0.7252773555369294, 1.0050017326824408, 0.7973628996802817, 0.7301480827117306, 0.7414581799560349, 0.7796038018339497, 0.7543694371356182, 0.7549990691380718, 0.8462791756287292, 0.7115511113106332, 0.8096164602126334, 0.6739672661202752, 0.7918784487925188, 0.7212704001367204, 0.7137628723488683, 0.7621903922698784, 0.7149434828447537, 0.7305304469239625, 0.780662818651215, 0.6794965807640578, 0.7182950364328065, 0.7034608339907338, 0.6916099531821254, 0.7713562070519593, 0.8084182337786897, 0.7558896202832739, 0.7955062599020424, 0.7957183118875921, 0.8462464528617613, 0.957015119038542, 0.7801496931415481, 0.7530682848414965, 0.770556532277029, 0.7608640893786329, 0.7360526014026403, 0.7560358338687209, 0.7698552462596476, 0.7636855617859514, 0.8060663191097219, 0.7558640239299197, 0.7934772967607623, 0.7364421069739608, 0.7277098188887638, 0.6980637689624367, 0.7498661168499484, 0.7613909732174675, 0.7357851394003128, 0.7518151329275135, 0.726970679911519, 0.7874477047574029, 0.6827263501117737, 0.8586427266996396, 0.7411717737795497, 0.7798670227687127, 0.7190337329885583, 0.8598103615204583, 0.7142628115553624, 0.6828984382385064, 0.7128858379356514, 0.737710006784675, 0.7883202834202601, 0.7707269951419746, 0.7179800845402116, 0.7183184166437322, 0.8277025386208521, 0.7691388923177562, 0.7710868360233161, 0.7252460152744247, 0.7331519809076898, 0.7197885654647329, 0.7963599552298034, 0.7257895775519879, 0.70366197608122, 0.8392011793092633, 0.8157439633038075, 0.8824026579892765, 0.7748007104880618, 0.7960135313509804, 0.6823127486280665, 0.8211328958106444, 0.765564520155139, 0.7633974263364933, 0.7115581629918288, 0.7649177436539857, 0.8028125396417518, 0.722663770244002, 0.7817470609949482, 0.7754108314368373, 0.6909469755051867, 0.7519944490400632, 0.7398622779337235, 0.8327983072121836, 0.8479552660987508, 0.710968597776179, 0.7265624959623862, 0.729264291738005, 0.7322052725774497, 0.6801587977799142, 0.888993934935906, 0.7328193582458189]
    },
    {
        'diff': [3.480478659230811, 2.733259332400142, 2.53210629116717, 2.6169222262920124, 2.5524288780909057, 2.5009590545802096, 4.3814563323671285, 3.242503747831939, 1.9801537737554769, 1.718107889222399, 2.8484890288257567, 2.967182818512831, 2.141919958481565, 2.6258022937203975, 2.037920673205327, 2.2216461688859854, 2.422527857067891, 2.4965631639542494, 2.712432815326295, 3.48985361942826, 2.308531500581134, 2.785237737622253, 3.626062107015397, 3.567532271506403, 2.2517818294236838, 2.763462775771912, 2.4193327947087515, 2.6826177969190463, 3.190436419428229, 2.987288913901776, 2.757065902201959, 2.2664641238943917, 2.3153730293522203, 2.496378624412522, 2.2068541352989897, 3.0068904943975134, 2.283192971885189, 2.2808903395258198, 2.2498800654285636, 2.410995711574508, 3.2891013098391424, 2.532748032325731, 3.3064750534192116, 2.7804457040742263, 2.3503377247081594, 1.8837086586726057, 3.2714934270528913, 3.6988762194735676, 3.224999714165783, 2.050507896331647, 2.57693731650649, 1.7526352674909345, 3.033160061956642, 2.9469780976479054, 2.319423574822703, 2.3887732154181474, 2.345269961189639, 1.9047680749217852, 3.007953810867422, 1.9330931130908058, 2.5479746489045634, 2.121641112810571, 3.190019096696024, 3.9315239167283758, 3.0311420744688955, 1.512461074641302, 2.970001122479051, 2.960133177885982, 2.2203517498024765, 2.323402799286001, 2.818085020349195, 2.6908560726615836, 3.2531402498115227, 2.974914146313548, 3.1205476402953622, 2.302846506085463, 2.4220761106490234, 2.407525918581147, 2.455933287515933, 1.4494780279150579, 2.0044478959360466, 2.0824853192309623, 2.6816907519288673, 3.4154228876132593, 2.6342127191593097, 2.6763715884013197, 3.043684714222543, 2.4269017945471836, 2.0824974610573124, 2.3701069203675966, 2.5178857992464465, 2.684166604672974, 3.420806597701815, 2.756191555246631, 2.396051674795887, 3.2963730584801962, 2.286773393603709, 2.380134047260606, 3.2544761138672307, 2.790026060455598],
        'dis': [30.48317943104495, 31.80569301839602, 23.412188158615432, 27.473375826114726, 25.39334500656853, 28.55614130458254, 32.257773186200424, 30.788751908826015, 21.54138717291421, 24.624164733647692, 27.60662018245967, 23.19037284989173, 27.015352865573647, 31.36186448162653, 21.034679437350004, 28.356712184630908, 25.864415830605147, 25.002880591933366, 23.905719656393565, 30.597137772117037, 24.93275611019709, 25.91838544574129, 32.885820287247206, 30.671635392212103, 25.113753311366207, 31.31148576992366, 21.346413727226377, 29.68993042072668, 28.715440761184762, 29.195597116499044, 23.12021288781412, 26.23440141213636, 29.25069359501234, 26.992408339333053, 20.26879071778836, 31.19077917082194, 23.065427553277683, 24.878857498200126, 30.181666009060336, 24.00686605563831, 36.02356484803575, 30.972960445070388, 29.0494162800362, 24.035523010920816, 25.61798016192838, 24.982613573066352, 30.344037639044693, 30.93701569456895, 30.611096174393094, 26.580789367440897, 27.02756518202021, 20.916637588181988, 28.35258735545026, 26.412805533054396, 21.654169512302282, 25.545135176430293, 21.232851848917726, 24.507164051049546, 30.948650581037537, 21.61485845010031, 26.91395807856317, 22.01006447798849, 25.391926376275638, 34.8523485859299, 34.54792186806098, 18.71885982375773, 28.22236183620284, 24.74675656782193, 25.5939716729294, 26.04798190088879, 27.940919902764982, 31.439116197858805, 31.277147442809383, 31.977578668317623, 25.516566136749578, 21.68587433770576, 23.822004415451183, 22.592843777681914, 27.802768669214863, 18.795429820682735, 24.16814275976866, 16.949787760579756, 23.90427688969393, 32.81042265049595, 24.174944746329526, 26.641805629338865, 31.02020650693807, 22.28888551505108, 23.205534031282983, 22.759264094248188, 30.2200864544299, 28.40815728892317, 26.161054343268606, 23.521820585036476, 23.28639883307652, 28.832352395489803, 26.7155291361597, 24.035476941265266, 32.21037089011423, 24.103240505788868],
        'max_diff': [0.22418266776374982, 0.16827973417224573, 0.30191779333703783, 0.21006884763453104, 0.19508025968830733, 0.26485750552621395, 0.3779426202014162, 0.314500528276877, 0.207740156578057, 0.12681657499895893, 0.24684163971259213, 0.2577892744465289, 0.3200450784697908, 0.2674172503053848, 0.24691034590395455, 0.2723432740096281, 0.20813489526439355, 0.29504919168130295, 0.3033242872865789, 0.2354605358492703, 0.24612841279720243, 0.2674289008617641, 0.5544334941124505, 0.27051117478142495, 0.14030813527423946, 0.2035398955066463, 0.2995708041363323, 0.21127058092083503, 0.24543738591342054, 0.21174574516638744, 0.27242841848114674, 0.19563230100541285, 0.24500961887006767, 0.28381107300001096, 0.21291081428358405, 0.19831460360971453, 0.23792090624931506, 0.2574280353487294, 0.23479727794592753, 0.23749905561077878, 0.3075068044286743, 0.14706385593398552, 0.25568783694292196, 0.21369312302413368, 0.15437073923692024, 0.12339913602951413, 0.4961709254946065, 0.3907301979370055, 0.2629888693429556, 0.12761111266193703, 0.20423398860947362, 0.17643737569436135, 0.2553353331153789, 0.25093842973004776, 0.19948728760120527, 0.17097775677353733, 0.2996844784794267, 0.1220032393996775, 0.44208209725099934, 0.20794155006308612, 0.1604665771817826, 0.19262604957591334, 0.2541210258613886, 0.39067463028604243, 0.14234444216410846, 0.12048204467469903, 0.24435142000529617, 0.3340478547453407, 0.19773537678196673, 0.19622783554040035, 0.24093211051972205, 0.2352475130800736, 0.2619972157204158, 0.24477959710509042, 0.2665545988180491, 0.23422148052463976, 0.2568633192333061, 0.24289848804005443, 0.34007163954821484, 0.11701558222608455, 0.21924528883442074, 0.15117942491975578, 0.20828771459350492, 0.21014788666073916, 0.22561248699993852, 0.22583104150756445, 0.2987210018886238, 0.31346979676964626, 0.10569469947139076, 0.2397307273426622, 0.1590550236731394, 0.20820726786907237, 0.22952789017924125, 0.16932015086476904, 0.13416568560667097, 0.2608879645400871, 0.1329981064919804, 0.22270453291140813, 0.2379776947742399, 0.27917158464540737],
        'max_dis': [0.9044654176474154, 0.9697363662656292, 0.759252133994349, 0.8725312554013265, 0.8261359724513391, 0.9786986886767302, 0.8195879045237766, 0.9441263104383482, 0.8960799920678967, 0.843364382417606, 0.7020139448859462, 0.932410904105083, 0.6058741080039132, 0.9004196017127606, 0.7916179523419582, 0.7727872671122058, 1.024593155686463, 0.8859891943104911, 0.6837245584291697, 0.75582443983382, 0.7918976153536109, 0.8508840863544886, 0.9306556881282666, 0.9077111132920608, 0.8300991604388835, 0.8569859747444317, 0.7499617351516497, 0.9509667811348752, 0.8847676722868821, 0.9819929667261695, 0.8102768164070768, 0.8364238958581209, 0.8322588038368022, 0.8615171811657201, 0.824250861617635, 0.992282143305221, 0.7971079755624082, 0.9807247740045426, 0.8831307783498581, 0.7780067766648809, 1.0209567658342427, 0.7854126436517737, 0.8610036951921238, 0.8246813989672203, 0.5992172635501425, 0.8946480416802791, 0.8049167378221407, 0.8375492518461172, 0.8770003951823391, 0.9390509037111692, 0.8445200653030758, 0.6729999591829992, 0.9014590193562071, 0.8555836179466393, 0.8456831425542726, 0.9881515161698489, 0.47507401145716677, 0.7827125490539808, 0.8521763368927138, 0.8401154442466933, 0.7974602588360084, 0.8336343793667029, 0.8585370309767268, 0.8785607059448548, 0.7975069261559231, 0.7824639729088894, 0.8963899361336704, 0.8185138534421309, 0.8557479367781803, 0.9690203221459801, 0.8687520219450949, 0.8942046440453916, 0.889001787474377, 0.8613395285581462, 0.8118583751021243, 0.6791194547749275, 0.8660966487747843, 0.6709718010386228, 0.9975872260412909, 0.7197655113618375, 0.907618875167582, 0.5808173162802535, 0.8623415335368535, 0.8619400201729444, 0.853670591592473, 0.8817292618056952, 0.8930633385371377, 0.757682687237134, 0.7285230791638966, 0.6803204779235346, 0.9524316653639083, 1.0013874392505815, 0.964457867300867, 0.8211282569209756, 0.8430220285364972, 0.9546954280778018, 0.9132323726534377, 0.9420543612839537, 0.8821702645500133, 0.7959043988330409]
    },
    {
        'diff': [2.4698115795858135, 2.50489108896705, 3.316894826797421, 2.40321612918729, 2.3582534856840978, 2.2148025461063323, 2.4241614392771424, 2.718243676070296, 2.279985362518687, 2.3923824815023798, 2.554669411843786, 2.788089167865167, 2.812065729610029, 3.0202964245226647, 3.597097238797277, 2.4243490110288546, 2.9085914270707356, 2.0237771056279525, 3.341171783563904, 2.8789891678441704, 1.863330998176919, 3.0608021686405014, 2.1733508284832963, 2.214119851476091, 2.0651089579044912, 2.2262588905817955, 1.8767261952206515, 1.917551979830874, 2.4868634841551955, 2.4821090981203873, 2.6854304700874856, 1.8212487171668263, 2.050214505559157, 2.1469131650554556, 2.5698177496517514, 2.490822923540595, 3.111799808006275, 3.35325534054044, 1.6952942384345566, 2.2611761996757167, 2.2295972648404137, 1.9388637891429317, 2.5332055513644693, 1.9014887843920263, 2.228559306325853, 2.69685996120957, 2.205950251981985, 3.478591200544006, 2.2192426292699006, 3.1998210345933766, 1.704009550246645, 2.545688534173814, 2.176102767254064, 2.130976402782576, 2.034074455293557, 2.667134824265413, 2.6098285508960894, 2.6265684844134234, 2.301994541384147, 4.339200160094171, 2.337262510793281, 2.966223799736592, 2.8153182624800928, 2.5854973840784687, 2.270345783593036, 2.291308030324745, 2.174529736500828, 2.6987345389998834, 3.3754629202402886, 2.8028151678708007, 2.952018190735359, 3.7347025940905634, 3.0094189947888186, 3.015871264365873, 2.1880273240802826, 2.691092041165391, 1.6087272622214588, 3.1140067047480517, 3.730746613923695, 1.993875996611984, 1.8799983200153128, 2.364719753117705, 2.3416741010091613, 2.29845662916203, 3.0689316935556854, 2.7658286741491436, 3.2083758706529366, 2.228107286965741, 2.001448161776774, 2.4296336000090495, 2.021895979810305, 2.7361366776173566, 2.1605639833511985, 2.072811628090375, 2.706840473848554, 2.568283739099708, 2.491655703130018, 2.141278321660911, 3.2515188936527277, 2.239396746799854],
        'dis': [24.985272871337628, 25.283699296048304, 28.455705978627766, 24.65301919298849, 25.779646154684894, 24.349320312423867, 26.79520670844399, 25.15559276060855, 21.451370637422684, 25.72149078812882, 25.02147317475438, 25.88471763396676, 26.371330709148896, 25.203152320182554, 31.95047512529382, 24.71888218659885, 26.276680331800307, 27.659672596832387, 28.98876656670174, 27.24565805683481, 21.314828082740878, 24.85650230639553, 24.79909915971622, 23.706549391178637, 25.709884566209197, 25.577116665287154, 26.512509589121944, 22.720419197021464, 29.584508091951815, 25.264964754276875, 24.3565167466945, 24.754694708359832, 24.17290304310588, 23.445106409969753, 26.801465658702057, 29.628422416605936, 25.709465759477638, 28.487598524524245, 23.836595106938447, 24.58246726895655, 30.026171314911537, 22.528154649617743, 25.050042763243106, 23.412005481724997, 24.50359059411498, 32.65147082959144, 25.037272818086564, 31.29386593332008, 26.014247894271875, 29.88730424016727, 24.315821654683617, 25.170541641818364, 26.017153065816423, 26.941294573721674, 25.302446596633917, 23.866837817359947, 27.371361477441756, 25.674130874526824, 24.592824850046636, 37.033982205391965, 24.189594050857643, 23.92049644038024, 29.569791048089698, 32.019507670717324, 24.431678471358847, 27.305610526462146, 26.134076962651942, 25.692455584508423, 36.2692741219128, 27.887389714150082, 24.965778213783782, 31.43134836411219, 26.518850769402945, 28.49812091086967, 25.09655660281267, 26.769440405714096, 24.070710977380617, 27.444351512019697, 31.38538492026731, 26.875281847819537, 25.800131684277034, 28.92921901787693, 28.022557707432806, 24.784923063559873, 25.576880756310356, 25.450284777672433, 28.881219078501097, 28.311581991508657, 25.840461505306802, 28.029290105731775, 23.81657302752776, 27.695267571215556, 22.355826192806212, 24.149743835303692, 28.370313566756387, 28.86499291479995, 24.66365587843938, 21.319970282048356, 28.805417494537565, 25.604161999638727],
        'max_diff': [0.20411480584809083, 0.2621937104012235, 0.32452115317149355, 0.23494579550286976, 0.3092148924718492, 0.34022467282110247, 0.19697389047445268, 0.25399273839087533, 0.22823710680752607, 0.19555683811374736, 0.3278606303830318, 0.19036299051006167, 0.26159844979358915, 0.2853141763126317, 0.30500451136847684, 0.3427924801547877, 0.3297115818827978, 0.19093261129981798, 0.30418480494881084, 0.2924221407344354, 0.22708693446036188, 0.2896551098715284, 0.17524970173705468, 0.22452531968173384, 0.2057244514704757, 0.16331397852397142, 0.2563366149168904, 0.26742846398154185, 0.20756651547282057, 0.31895779114980627, 0.2237735148933928, 0.1653660450295189, 0.1932923502550703, 0.18050040394098987, 0.2017886814629346, 0.15863716802197303, 0.29648516326853513, 0.31219590949722864, 0.16732442451009533, 0.18899029020335245, 0.25957102089341677, 0.19191859026776337, 0.24497618738919646, 0.16900475598224718, 0.15631544080731308, 0.20588591608317586, 0.18343606952387193, 0.29056456907354766, 0.22512441176387377, 0.37933830816083214, 0.21115014423069, 0.19540777877843876, 0.2149646228454447, 0.2717873116287278, 0.17373524588413192, 0.20085444145985898, 0.2524431667898528, 0.30205780343565447, 0.2830585002141223, 0.2720269468353984, 0.2257042500582016, 0.27083710731755695, 0.3253716414447827, 0.19271962470101167, 0.22463873034029058, 0.21491372146059762, 0.1527654237821533, 0.21014920001886317, 0.2292135500114587, 0.22459249983849583, 0.25911832618312935, 0.3531291051941654, 0.2963702141807909, 0.2844248421881539, 0.24246408396687436, 0.27555856389275973, 0.185922182077205, 0.258620476143522, 0.30745540091347323, 0.16312650370482207, 0.1559036784856329, 0.16212953487990506, 0.23651595674270287, 0.20344570379647414, 0.35600807303881254, 0.2808826043193673, 0.23092965749316852, 0.21563704250693982, 0.15137633859850697, 0.19987713734311408, 0.17819908136209373, 0.2980157477854394, 0.20105200541760593, 0.18272106491041054, 0.20495247112675125, 0.15521854133105895, 0.20670877811292465, 0.17723149567486374, 0.34016464751840647, 0.23611953523886653],
        'max_dis': [0.9309651059382256, 0.7971412770344606, 0.8565801494460142, 0.7844234507917196, 0.9178642026693329, 0.9915358754393719, 0.9087921259951894, 0.850125828140993, 0.7766773605741952, 0.9179058117542753, 0.9990653363144424, 0.7967906320399232, 0.9018756178594682, 0.8116149105791852, 0.8103289512636532, 0.9680677618079865, 0.898143834661483, 0.8421414940559561, 0.9187366470252127, 0.877887898136136, 0.9285716544352252, 0.863332421477031, 0.8702687109945275, 0.8470972824504359, 0.8824229483362754, 0.9045495727124274, 0.90743796859065, 0.8529219148759337, 0.8735640335904608, 0.8384725850691189, 0.7799930849344114, 0.8543584676758487, 0.7433073338252665, 0.891464727632066, 0.902709985977668, 0.8386735453268681, 0.900399611986838, 0.8435952265006337, 0.8795527601881529, 0.7659447318667352, 1.0572058261421236, 0.8203242956824714, 0.8342007284884425, 0.8025224543469387, 0.8770324485329645, 0.903389367011022, 0.7952787419165366, 0.7910297460206579, 0.9130805207351272, 0.7747225156863926, 0.9133034381546707, 0.8445507401151662, 1.0372450724174, 0.8435747341694865, 0.8411408396936352, 0.7959547418485786, 0.8320689976196954, 0.8231233457481104, 0.9420663922214616, 0.9374146071938094, 0.8247620309637462, 0.7472020574144171, 0.8317179553544093, 0.8953169716429032, 0.896298248958108, 0.7980115732073539, 0.9164338316264624, 0.8466840765605856, 0.7950720383121775, 0.8133703385103832, 0.8656167631836783, 0.8689957015159165, 0.9466574014908875, 0.8884138433703227, 0.8891121543289032, 0.7995386674134353, 1.026324040983201, 0.9342835324293355, 0.8397124806235597, 0.8753572436909562, 0.8533320285907298, 1.0051353940788337, 0.9027181070174587, 0.8800557992825813, 0.8327138147600166, 0.8744761117756628, 0.8733151485535415, 0.8396289190635503, 0.9357337054064199, 0.9119764870878437, 0.8766649068353016, 0.8758833326761988, 0.8333448891503001, 0.7695787830064865, 0.8962331899942758, 0.8316209751569916, 0.8261317883014011, 0.7544649954051369, 0.9058284710037529, 0.9062041943090182]
    }
]

In [ ]:
clean_train = []
clean_train_label = []
for idx, data in enumerate(clean):
    for i, d in enumerate(data['diff']):
        clean_train.append([data['diff'][i], data['dis'][i], data['max_diff'][i], data['max_dis'][i]])
        clean_train_label.append(0)

In [ ]:
poisoned_train = []
poisoned_train_label = []
for idx, data in enumerate(poisoned):
    for i, d in enumerate(data['diff']):
        poisoned_train.append([data['diff'][i], data['dis'][i], data['max_diff'][i], data['max_dis'][i]])
        poisoned_train_label.append(1)

In [ ]:
print(clean_train [:5])

In [ ]:
X_train = clean_train + poisoned_train
len(X_train)

In [ ]:
y_train = clean_train_label + poisoned_train_label

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train,y_train, random_state=0)
print(y_train[:5])

# Perceptron

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import numpy as np

accuracy=[]
recall=[]

for i in range(50):
    X_train = clean_train + poisoned_train
    y_train = clean_train_label + poisoned_train_label
    clf = Perceptron(tol=1e-3, early_stopping=True, random_state=0)
    clf.fit(X_train, y_train)
    X_test = clean_test + poisoned_test
    y_test = clean_test_label + poisoned_test_label
    accuracy.append(clf.score(X_test, y_test))
    y_pred = clf.predict(X_test)
    #print(metrics.confusion_matrix(y_test,y_pred,labels=[0,1]))
    recall.append(metrics.recall_score(y_test, y_pred))
        
print(accuracy)
print(recall)

In [ ]:
np.mean(accuracy)

In [ ]:
np.mean(recall)

# SVM scaled


In [ ]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import sklearn.metrics as metrics

accuracy = []
recall = []

for i in range(50):
    X_train = clean_train + poisoned_train
    y_train = clean_train_label + poisoned_train_label
  # Always scale the input. The most convenient way is to use a pipeline.
    clf = make_pipeline(StandardScaler(),SGDClassifier(loss="hinge", max_iter=1000, tol=1e-3))
    clf.fit(X_train, y_train)
    X_test = clean_test + poisoned_test
    y_test = clean_test_label + poisoned_test_label
    accuracy.append(clf.score(X_test, y_test))
    y_pred = clf.predict(X_test)
    #print(metrics.confusion_matrix(y_test,y_pred,labels=[0,1]))
    recall.append(metrics.recall_score(y_test, y_pred))

print(accuracy)
print(recall)

In [ ]:
np.mean(accuracy)

In [ ]:
np.mean(recall)

# SVM 2 

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

accuracy = []
recall=[]

for i in range(50):
    X_train = clean_train + poisoned_train
    y_train = clean_train_label + poisoned_train_label
  # Always scale the input. The most convenient way is to use a pipeline.
    clf = make_pipeline(SGDClassifier(loss="hinge",max_iter=1000, tol=1e-3))
    clf.fit(X_train, y_train)
    X_test = clean_test + poisoned_test
    y_test = clean_test_label + poisoned_test_label
    accuracy.append(clf.score(X_test, y_test))
    y_pred = clf.predict(X_test)
    recall.append(metrics.recall_score(y_test, y_pred))

print(accuracy)
print(recall)


In [ ]:
np.mean(accuracy)

In [ ]:
np.mean(recall)

# SVM 3

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

accuracy = []
recall = []

for i in range(50):
    X_train = clean_train + poisoned_train
    y_train = clean_train_label + poisoned_train_label
  # Always scale the input. The most convenient way is to use a pipeline.
    clf = make_pipeline(StandardScaler(),
                        SVC(gamma='scale'))
    clf.fit(X_train, y_train)
    
    X_test = clean_test + poisoned_test
    y_test = clean_test_label + poisoned_test_label
    accuracy.append(clf.score(X_test, y_test))
    y_pred = clf.predict(X_test)
    recall.append(metrics.recall_score(y_test, y_pred))

print(accuracy)
print(recall)

In [ ]:
np.mean(accuracy)

In [ ]:
np.mean(recall)